In [1]:
import torch
import torch.nn as nn

import clip
import numpy as np


### 1. CLIP

In [ ]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

In [9]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [22]:
model, preprocess = clip.load("ViT-B/32")
input_resolution = model.visual.input_resolution

print("Input resolution:", input_resolution)

Input resolution: 224


In [ ]:
trans = model.transformer
trans

### 2. Data

#### 2.0.0 CSI $\rightarrow$ CSI-Ratio

In [2]:
import numpy as np
import math

# 假设CSI数据的形状为 (num_packets, num_antennas, num_subcarriers)
# num_packets: 数据包数量（时间序列）
# num_antennas: 天线数量（3根天线）
# num_subcarriers: 子载波数量（30个子载波）

def calculate_amplitude_variance_ratio(csi_data):
    """
    计算每个天线的所有子载波的幅度方差比之和
    :param csi_data: CSI数据，形状为 (num_packets, num_antennas, num_subcarriers)
    :return: 每个天线的幅度方差比之和，形状为 (num_antennas,)
    """
    # 计算幅度
    amplitude = np.abs(csi_data)  # 形状为 (num_packets, num_antennas, num_subcarriers)

    # 计算每个天线和子载波的幅度方差
    variance = np.var(amplitude, axis=0)  # 形状为 (num_antennas, num_subcarriers)

    # 计算每个天线的幅度方差比之和
    variance_ratio_sum = np.sum(variance, axis=1)  # 形状为 (num_antennas,)

    return variance_ratio_sum

def select_reference_antenna(csi_data):
    """
    选择参考天线（幅度方差比之和最小的天线）
    :param csi_data: CSI数据，形状为 (num_packets, num_antennas, num_subcarriers)
    :return: 参考天线的索引
    """
    variance_ratio_sum = calculate_amplitude_variance_ratio(csi_data)
    reference_antenna_index = np.argmin(variance_ratio_sum)
    return reference_antenna_index

def calculate_csi_ratio(csi_data):
    """
    计算CSI比值（CSI-Ratio）
    :param csi_data: CSI数据，形状为 (num_packets, num_antennas, num_subcarriers)
    :return: CSI比值数据，形状为 (num_packets, num_antennas, num_subcarriers)
    """
    num_packets, num_antennas, num_subcarriers = csi_data.shape

    # 选择参考天线
    reference_antenna_index = select_reference_antenna(csi_data)
    reference_csi = csi_data[:, reference_antenna_index, :]  # 参考天线的CSI数据

    # 计算CSI比值
    csi_ratio = np.zeros_like(csi_data)
    for antenna in range(num_antennas):
        if antenna == reference_antenna_index:
            # 参考天线的CSI比值为1
            csi_ratio[:, antenna, :] = 1.0
        else:
            # 其他天线的CSI比值
            csi_ratio[:, antenna, :] = csi_data[:, antenna, :] / reference_csi

    return csi_ratio

In [ ]:
# 示例：生成随机CSI数据
num_packets = 100  # 假设有100个数据包
num_antennas = 3   # 3根天线
num_subcarriers = 30  # 30个子载波
csi_data = np.random.rand(num_packets, num_antennas, num_subcarriers) + 1j * np.random.rand(num_packets, num_antennas, num_subcarriers)

# 计算CSI比值
csi_ratio = calculate_csi_ratio(csi_data)

# 输出结果
print("CSI比值数据的形状:", csi_ratio.shape)

#### 2.0.1 CSI Resample

In [ ]:
import numpy as np
from scipy.interpolate import interp1d

def resample_csi_sequence(csi_sequence, target_length=500):
    """
    对单个CSI序列进行重采样至目标长度
    :param csi_sequence: 原始CSI序列，形状为 (original_length, num_antennas, num_subcarriers)
    :param target_length: 目标序列长度（默认为500）
    :return: 重采样后的CSI序列，形状为 (target_length, num_antennas, num_subcarriers)
    """
    original_length = csi_sequence.shape[0]
    num_antennas = csi_sequence.shape[1]
    num_subcarriers = csi_sequence.shape[2]
    
    # 创建新时间轴
    original_time = np.linspace(0, 1, original_length)  # 归一化时间轴
    new_time = np.linspace(0, 1, target_length)          # 目标时间轴
    
    # 初始化输出序列
    resampled_sequence = np.zeros((target_length, num_antennas, num_subcarriers), dtype=np.complex_)
    
    # 对每个天线和子载波进行插值
    for ant in range(num_antennas):
        for sc in range(num_subcarriers):
            # 提取原始复数值序列
            complex_sequence = csi_sequence[:, ant, sc]
            
            # 创建插值函数（对实部和虚部分别插值）
            interp_real = interp1d(original_time, complex_sequence.real, kind='linear')
            interp_imag = interp1d(original_time, complex_sequence.imag, kind='linear')
            
            # 生成新序列
            resampled_real = interp_real(new_time)
            resampled_imag = interp_imag(new_time)
            
            # 组合复数值
            resampled_sequence[:, ant, sc] = resampled_real + 1j * resampled_imag
            
    return resampled_sequence

def batch_resample(csi_data, target_length=500):
    """
    批量处理CSI数据集
    :param csi_data: 原始CSI数据集，形状为 (num_samples, original_length, num_antennas, num_subcarriers)
    :param target_length: 目标序列长度
    :return: 重采样后的数据集，形状为 (num_samples, target_length, num_antennas, num_subcarriers)
    """
    num_samples = csi_data.shape[0]
    resampled_data = []
    
    for i in range(num_samples):
        resampled_seq = resample_csi_sequence(csi_data[i], target_length)
        resampled_data.append(resampled_seq)
    
    return np.array(resampled_data)

In [ ]:
# 生成示例数据
np.random.seed(0)

# 创建不同长度的CSI序列
raw_data = [
    np.random.randn(300, 3, 30) + 1j*np.random.randn(300, 3, 30),  # 长度300
    np.random.randn(750, 3, 30) + 1j*np.random.randn(750, 3, 30),  # 长度750
    np.random.randn(480, 3, 30) + 1j*np.random.randn(480, 3, 30)   # 长度480
]

# 执行重采样
resampled_data = batch_resample(np.array(raw_data, dtype=object), 500)

# 验证输出形状
print("原始序列长度:", [seq.shape[0] for seq in raw_data])
print("重采样后长度:", [seq.shape[0] for seq in resampled_data])
print("示例数据形状（第一个样本）:", resampled_data[0].shape)

#### 2.1 CSI Dataset 

In [1]:
import scipy.io as sio
import os
import numpy as np
from typing import List
import random
import pywt

from dataset.data import get_csi_data
from dataset.datasets import CSI_Dataset, HAR_Dataset
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader

In [2]:
# Widar
data_path = '../data'
batch_size = 20
time_length = 2000
req_domains = ['users', 'gesture_type', 'tosor_loc', 'face_ori', 'rec_dev']

In [ ]:
train_datas, train_labels, eval_datas, eval_labels = get_csi_data(
    data_path, 
    req_domains=req_domains,
)

train_labels.shape

In [4]:
# HAR
train_datas = HAR_Dataset('D:/NTU-Fi_HAR/train_amp', time_length=2000)

#### 2.2 DFS Data

In [1]:
import scipy.io as sio
import os
import numpy as np
from typing import List
import random
import pywt

from dataset.data import get_dfs_data
from dataset.datasets import DFS_Dataset
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader

In [2]:
dfs_data_path = '../dfs_data'
req_domains = ['users', 'gesture_type', 'tosor_loc', 'face_ori']

In [3]:
train_datas, train_labels, eval_datas, eval_labels = get_dfs_data(dfs_data_path, req_domains)

In [4]:
train_dataset = DFS_Dataset(
    dfs_data_path,
    train_datas,
    train_labels
)

### 2.3 BVP Data

In [1]:
import scipy.io as sio
import os
import numpy as np
from typing import List

from dataset.data import get_bvp_data

from dataset.datasets import CSI_Dataset, HAR_Dataset, BVP_Dataset
from torch.utils.data import DataLoader

In [2]:
train_datas, train_labels, eval_datas, eval_labels = get_bvp_data(
    '../bvp_data', 
    req_domains=['users', 'gesture_type', 'tosor_loc', 'face_ori']
)

In [3]:
data = BVP_Dataset('../bvp_data', train_datas, train_labels)
train_loader = DataLoader(data, batch_size=8, shuffle=True)

### 3.Model

In [1]:
from model.embed import TokenEmbedding

from einops import rearrange
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary


d:\Anaconda\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### GPT2

In [ ]:
import torch
from transformers.models.gpt2.modeling_gpt2 import GPT2Model

In [ ]:
# GPT2 Model
model = GPT2Model.from_pretrained('gpt2')

### Qwen2.5-3B

In [2]:
import torch
from transformers import Qwen2ForCausalLM, Qwen2Model, Qwen2Config

from peft import get_peft_model, LoraConfig, TaskType

d:\Anaconda\envs\Deep_Learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_config = Qwen2Config.from_pretrained('unsloth/Qwen2.5-3B')
model_config.num_hidden_layers = 12

In [ ]:
model_config

In [4]:
model = Qwen2Model.from_pretrained('unsloth/Qwen2.5-3B', config=model_config)

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.54s/it]


In [5]:
model

Qwen2Model(
  (embed_tokens): Embedding(151936, 2048, padding_idx=151654)
  (layers): ModuleList(
    (0-11): 12 x Qwen2DecoderLayer(
      (self_attn): Qwen2SdpaAttention(
        (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
        (k_proj): Linear(in_features=2048, out_features=256, bias=True)
        (v_proj): Linear(in_features=2048, out_features=256, bias=True)
        (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        (rotary_emb): Qwen2RotaryEmbedding()
      )
      (mlp): Qwen2MLP(
        (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
        (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
        (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
    )
  )
  (norm): Qwen2RMSNorm((2048,), eps=1e-06)
  (rotary_emb): Q

In [11]:
model.get_input_embeddings().weight.shape

torch.Size([128256, 2048])

#### Llama-3-8B

In [8]:
import torch
from transformers import LlamaModel, LlamaConfig

In [9]:
model = LlamaModel.from_pretrained('unsloth/Llama-3.2-1B')

In [ ]:
model_config = LlamaConfig.from_pretrained('unsloth/Llama-3.2-1B')
model_config

In [10]:
model

LlamaModel(
  (embed_tokens): Embedding(128256, 2048, padding_idx=128004)
  (layers): ModuleList(
    (0-15): 16 x LlamaDecoderLayer(
      (self_attn): LlamaSdpaAttention(
        (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
        (k_proj): Linear(in_features=2048, out_features=512, bias=False)
        (v_proj): Linear(in_features=2048, out_features=512, bias=False)
        (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        (rotary_emb): LlamaRotaryEmbedding()
      )
      (mlp): LlamaMLP(
        (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
        (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
        (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
    )
  )
  (norm): LlamaRMSNorm((2048,), eps=1e-05)
  (rotary_emb): L